### Identifying Entities in Healthcare Data

In [1]:
# Import libraries and packages

!pip install pycrf
!pip install sklearn-crfsuite
!pip install spacy

import spacy
import sklearn_crfsuite
from sklearn_crfsuite import metrics

model = spacy.load("en_core_web_sm")

### Data Preprocessing - Convert word per line to sentences

In [2]:
## Read the datasets

# Reading the train and test sentences and labels
with open('train_sent', 'r') as train_sent_file:
  train_words = train_sent_file.readlines()

with open('train_label', 'r') as train_labels_file:
  train_labels_by_word = train_labels_file.readlines()

with open('test_sent', 'r') as test_sent_file:
  test_words = test_sent_file.readlines()

with open('test_label', 'r') as test_labels_file:
  test_labels_by_word = test_labels_file.readlines()

In [3]:
# Count the number or tokens and labels - they should match
print("Count of tokens in training set\n","No. of words: ",len(train_words),"\nNo. of labels: ",len(train_labels_by_word))
print("\n\nCount of tokens in test set\n","No. of words: ",len(test_words),"\nNo. of labels: ",len(test_labels_by_word))

Count of tokens in training set
 No. of words:  48501 
No. of labels:  48501


Count of tokens in test set
 No. of words:  19674 
No. of labels:  19674


In [4]:
# Creat a function to combine tokens into sentences separated by "\n"
def convert_to_sentences(dataset):
    sent_list = []
    sent = ""
    for entity in dataset:
        if entity != '\n':
            sent = sent + entity[:-1] + " "
        else: 
            sent_list.append(sent[:-1]) 
            sent = ""
    return sent_list

In [5]:
# Converting tokens to sentences and individual labels
train_sentences = convert_to_sentences(train_words)
train_labels = convert_to_sentences(train_labels_by_word)
test_sentences = convert_to_sentences(test_words)
test_labels = convert_to_sentences(test_labels_by_word)

In [6]:
## Print 5 training sentences and their corresponding labels

print("First five training sentences and their labels:\n")
for i in range(5):
    print(train_sentences[i],"\n",train_labels[i],"\n")

First five training sentences and their labels:

All live births > or = 23 weeks at the University of Vermont in 1995 ( n = 2395 ) were retrospectively analyzed for delivery route , indication for cesarean , gestational age , parity , and practice group ( to reflect risk status ) 
 O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 

The total cesarean rate was 14.4 % ( 344 of 2395 ) , and the primary rate was 11.4 % ( 244 of 2144 ) 
 O O O O O O O O O O O O O O O O O O O O O O O O O 

Abnormal presentation was the most common indication ( 25.6 % , 88 of 344 ) 
 O O O O O O O O O O O O O O O 

The `` corrected '' cesarean rate ( maternal-fetal medicine and transported patients excluded ) was 12.4 % ( 273 of 2194 ) , and the `` corrected '' primary rate was 9.6 % ( 190 of 1975 ) 
 O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 

Arrest of dilation was the most common indication in both `` corrected '' subgroups ( 23.4 a

In [7]:
## Print 5 test sentences and their corresponding labels

print("First five test sentences and their labels:\n")
for i in range(5):
    print(test_sentences[i],"\n",test_labels[i],"\n")

First five test sentences and their labels:

Furthermore , when all deliveries were analyzed , regardless of risk status but limited to gestational age > or = 36 weeks , the rates did not change ( 12.6 % , 280 of 2214 ; primary 9.2 % , 183 of 1994 ) 
 O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 

As the ambient temperature increases , there is an increase in insensible fluid loss and the potential for dehydration 
 O O O O O O O O O O O O O O O O O O O 

The daily high temperature ranged from 71 to 104 degrees F and AFI values ranged from 1.7 to 24.7 cm during the study period 
 O O O O O O O O O O O O O O O O O O O O O O O O 

There was a significant correlation between the 2- , 3- , and 4-day mean temperature and AFI , with the 4-day mean being the most significant ( r = 0.31 , p & # 60 ; 0.001 ) 
 O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 

Fluctuations in ambient temperature are inversely correlated to ch

### Count the number of lines of labels in the processed train and test dataset.

In [8]:
print("Number of lines of labels in the train dataset: {}".format(len(train_labels)))
print("Number of lines of labels in the test dataset: {}".format(len(test_labels)))

Number of lines of labels in the train dataset: 2599
Number of lines of labels in the test dataset: 1056


### Count the number of sentences in the processed train and test dataset

In [9]:
print("Number of sentences in the train dataset: {}".format(len(train_sentences)))
print("Number of sentences in the test dataset: {}".format(len(test_sentences)))

Number of sentences in the train dataset: 2599
Number of sentences in the test dataset: 1056


### Let's do the POS tagging to extract NOUN & PNOUN along with frequency

In [10]:
# Combined train and test sentences
combined_sent = train_sentences + test_sentences
print("Number of sentences in combined dataset (training + test): {}".format(len(combined_sent)))

Number of sentences in combined dataset (training + test): 3655


In [11]:
# List of tokens which have PoS tag of 'NOUN' or 'PROPN'
noun_propn = []         
pos_tag = []            
for sent in combined_sent:
    for token in model(sent):
        if token.pos_ in ['NOUN', 'PROPN']:
           noun_propn.append(token.text)
           pos_tag.append(token.pos_)
print("No. of tokens in combined dataset with PoS tag of 'NOUN' or 'PROPN': {}".format(len(noun_propn)))

No. of tokens in combined dataset with PoS tag of 'NOUN' or 'PROPN': 24373


### Print the top 25 most common tokens with NOUN or PROPN PoS tags

In [14]:
import pandas as pd
noun_pos = pd.DataFrame({"NOUN_PROPN":noun_propn,"POS_tag":pos_tag})
print("Top 25 comon tokens with PoS tag of 'NOUN' or 'PROPN' \n")
print(noun_pos["NOUN_PROPN"].value_counts().head(25))

Top 25 comon tokens with PoS tag of 'NOUN' or 'PROPN' 

patients        492
treatment       281
%               247
cancer          200
therapy         175
study           154
disease         142
cell            140
lung            116
group            94
chemotherapy     88
gene             87
effects          85
results          79
women            77
use              74
TO_SEE           74
risk             71
surgery          71
cases            71
analysis         70
rate             67
response         66
survival         65
children         64
Name: NOUN_PROPN, dtype: int64


### Features for CRF

In [17]:
sentence = train_sentences[1]   
sent_list = sentence.split()      
position = 2                      

word = sent_list[position]        # Extracting word for PoS tag analysis.

print(sentence)

print("\nPoS tag of word in isolation\nWord:",word,"--",model(word)[0].pos_,"\n")

print("PoS tag of all words in sentence with context in tact.")
for token in model(sentence):
    print(token.text, "--", token.pos_)

# Modified workflow to obtain PoS tag of specific word in question while keeping sentence context in tact.
print("\nResult of modified workflow to obtain PoS tag of word at a specific position while keeping context within sentence in-tact.")
cnt = 0
for token in model(sentence):
      postag = token.pos_
      if (token.text == word) and (cnt == position):
          break
      cnt += 1
print("Word:", word,"POSTAG:",postag)

The total cesarean rate was 14.4 % ( 344 of 2395 ) , and the primary rate was 11.4 % ( 244 of 2144 )

PoS tag of word in isolation
Word: cesarean -- VERB 

PoS tag of all words in sentence with context in tact.
The -- DET
total -- ADJ
cesarean -- ADJ
rate -- NOUN
was -- AUX
14.4 -- NUM
% -- NOUN
( -- PUNCT
344 -- NUM
of -- ADP
2395 -- NUM
) -- PUNCT
, -- PUNCT
and -- CCONJ
the -- DET
primary -- ADJ
rate -- NOUN
was -- AUX
11.4 -- NUM
% -- NOUN
( -- PUNCT
244 -- NUM
of -- ADP
2144 -- NUM
) -- PUNCT

Result of modified workflow to obtain PoS tag of word at a specific position while keeping context within sentence in-tact.
Word: cesarean POSTAG: ADJ


#### The work cesarean seems fine when part of sentence but incorrect individually

In [19]:
# Function to obtain contextual PoS tagger.
def contextual_pos_tagger(sent_list,position):
    '''Obtaining PoS tag for individual word with sentence context in-tact. 
       If the PoS tag is obtained for a word individually, it may not capture the context of use in the sentence and may assign the incorrect PoS tag.'''

    sentence = " ".join(sent_list)          
    posit = 0
    for token in model(sentence):
        postag = token.pos_
        if (token.text == word) and (posit == position):
            break
        posit += 1
    return postag

In [20]:
# Define the features to get the feature values for one word.
def getFeaturesForOneWord(sent_list, position):
  word = sent_list[position]
    
  # Obtaining features for current word
  features = [
    'word.lower=' + word.lower(),                                   
    'word.postag=' + contextual_pos_tagger(sent_list, position),    
    'word[-3:]=' + word[-3:],                                       
    'word[-2:]=' + word[-2:],                                       
    'word.isupper=%s' % word.isupper(),                             
    'word.isdigit=%s' % word.isdigit(),                             
    'words.startsWithCapital=%s' % word[0].isupper()                
  ]
 
  if(position > 0):
    prev_word = sent_list[position-1]
    features.extend([
    'prev_word.lower=' + prev_word.lower(),                               
    'prev_word.postag=' + contextual_pos_tagger(sent_list, position - 1), 
    'prev_word.isupper=%s' % prev_word.isupper(),                         
    'prev_word.isdigit=%s' % prev_word.isdigit(),                         
    'prev_words.startsWithCapital=%s' % prev_word[0].isupper()            
  ])
  else:
    features.append('BEG')                                                 
 
  if(position == len(sent_list)-1):
    features.append('END')                                                
 
  return features

### Getting the features

In [21]:
# Code to get features for a sentence.
def getFeaturesForOneSentence(sentence):
  sentence_list = sentence.split()
  return [getFeaturesForOneWord(sentence_list, position) for position in range(len(sentence_list))]

In [22]:
# Check feature extraction
example_sentence = train_sentences[5]
print(example_sentence)

features = getFeaturesForOneSentence(example_sentence)
features[0]

Cesarean rates at tertiary care hospitals should be compared with rates at community hospitals only after correcting for dissimilar patient groups or gestational age


['word.lower=cesarean',
 'word.postag=NOUN',
 'word[-3:]=ean',
 'word[-2:]=an',
 'word.isupper=False',
 'word.isdigit=False',
 'words.startsWithCapital=True',
 'BEG']

In [23]:
features[5]

['word.lower=hospitals',
 'word.postag=NOUN',
 'word[-3:]=als',
 'word[-2:]=ls',
 'word.isupper=False',
 'word.isdigit=False',
 'words.startsWithCapital=False',
 'prev_word.lower=care',
 'prev_word.postag=NOUN',
 'prev_word.isupper=False',
 'prev_word.isdigit=False',
 'prev_words.startsWithCapital=False']

In [24]:
# Code to get the labels for a sentence.
def getLabelsInListForOneSentence(labels):
  return labels.split()

In [25]:
# Check label extraction
example_labels = getLabelsInListForOneSentence(train_labels[5])
print(example_labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### Input and target variables

In [26]:
X_train = [getFeaturesForOneSentence(sentence) for sentence in train_sentences]
X_test = [getFeaturesForOneSentence(sentence) for sentence in test_sentences]

### Labels as the target variable for test and the train dataset

In [27]:
Y_train = [getLabelsInListForOneSentence(labels) for labels in train_labels]
Y_test = [getLabelsInListForOneSentence(labels) for labels in test_labels]

### CRF Model

In [31]:
# Build CRF model using 300 iterations
crf = sklearn_crfsuite.CRF(max_iterations=300)
try:
    crf.fit(X_train, Y_train)
except AttributeError:
    pass
predictions = crf.predict(X_test)

### Evaluation

#### Predict the labels of each of the tokens in each sentence of the test dataset that has been pre processed earlier.

In [32]:
Y_pred = crf.predict(X_test)

### f1 score using the actual labels and the predicted labels of the test dataset

In [33]:
metrics.flat_f1_score(Y_test, Y_pred, average='weighted')

0.908051252636922

#### -  f1 score of 91% seems pretty good

In [34]:
# Example test sentence and corresponding actual and predicted labels 
print("Sentence: ",test_sentences[15])
print("Actual labels:    ", Y_test[15])
print("Predicted labels: ", Y_pred[15])

Sentence:  The rate of severe preeclampsia was increased significantly in the triplet group 12 of 53 ( 22.6 % ) as compared with the twin group 3 of 53 ( 5.7 % ) ( OR = 4.9 , 95 % CI 1.2-23.5 , p = 0.02 )
Actual labels:     ['O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted labels:  ['O', 'O', 'O', 'D', 'D', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### Identifying Diseases and Treatments using Custom NER

In [45]:
# Extracting a dictionary of all the predicted diseases from our test data and the corresponding treatments.
disease_treatment = {}            
for i in range(len(Y_pred)):
    cnt_disease = 0           
    cnt_treatment = 0         
    diseases = [""]           
    treatment = [""]          
    length = len(Y_pred[i])   
    for j in range(length):
        # Checking for label indicating disease for current word ('D')
        if (Y_pred[i][j] == 'D'):                                                     
            diseases[cnt_disease] += (X_test[i][j][0].split('=')[1] + " ")            
            if j < length - 1:
                if (Y_pred[i][j+1] != 'D'):                                            
                    # If next word does not have label 'D', then truncate the space added at the end of the last word.
                    diseases[cnt_disease] = diseases[cnt_disease][:-1]
                    cnt_disease += 1
                    diseases.append("")                                               
            else:
                diseases[cnt_disease] = diseases[cnt_disease][:-1]
                cnt_disease += 1
                diseases.append("")
       # Checking for label indicating treatment for current word ('T')     
        if (Y_pred[i][j] == 'T'):                                                     
            treatment[cnt_treatment] += (X_test[i][j][0].split('=')[1] + " ")
            if j < length - 1:
                if (Y_pred[i][j+1] != 'T'):                                            
                    # If next word does not have label 'T', then truncate the space added at the end of the last word.
                    treatment[cnt_treatment] = treatment[cnt_treatment][:-1]
                    cnt_treatment += 1
                    treatment.append("")                                              
                treatment[cnt_treatment] = treatment[cnt_treatment][:-1]
                cnt_treatment += 1
                treatment.append("")

    diseases.pop(-1)    
    treatment.pop(-1)   
    
    if len(diseases) > 0:       
        for disease in diseases:
            if disease in disease_treatment.keys():
                # Extend treatment list if other treatments for the particular disease already exist
                disease_treatment[disease].extend(treatment)
            else:
                # Creating list of treatments for particular disease if it doesn not exist already.
                disease_treatment[disease] = treatment

In [46]:
# Displaying dictionary of extracted diseases and potential treatments.
disease_treatment = {}          
for i in range(len(Y_pred)):
    cnt_disease = 0           
    cnt_treatment = 0         
    diseases = [""]           
    treatment = [""]          
    length = len(Y_pred[i])   
    for j in range(length):
        # Checking for label indicating disease for current word ('D')
        if (Y_pred[i][j] == 'D'):                                                     
            diseases[cnt_disease] += (X_test[i][j][0].split('=')[1] + " ")            
            if j < length - 1:
                if (Y_pred[i][j+1] != 'D'):                                            
                    # If next word does not have label 'D', then truncate the space added at the end of the last word.
                    diseases[cnt_disease] = diseases[cnt_disease][:-1]
                    cnt_disease += 1
                    diseases.append("")                                               
            else:
                diseases[cnt_disease] = diseases[cnt_disease][:-1]
                cnt_disease += 1
                diseases.append("")
     # Checking for label indicating treatment for current word ('T')                       
        if (Y_pred[i][j] == 'T'):                                                     
            treatment[cnt_treatment] += (X_test[i][j][0].split('=')[1] + " ") 
            if j < length - 1:
                if (Y_pred[i][j+1] != 'T'):                                            
                    # If next word does not have label 'T', then truncate the space added at the end of the last word.
                    treatment[cnt_treatment] = treatment[cnt_treatment][:-1]
                    cnt_treatment += 1
                    treatment.append("")                                              
            else:
                treatment[cnt_treatment] = treatment[cnt_treatment][:-1]
                cnt_treatment += 1
                treatment.append("")

    diseases.pop(-1)    
    treatment.pop(-1)   
    
    if len(diseases) > 0:       
        for disease in diseases:
            if disease in disease_treatment.keys():
                # Extend treatment list if other treatments for the particular disease already exist
                disease_treatment[disease].extend(treatment)
            else:
                # Creating list of treatments for particular disease if it doesn not exist already.
                disease_treatment[disease] = treatment

In [47]:
# Cleaned version of "disease_treatment" dictionary
cleaned_dict = {}
for key in disease_treatment.keys():
    if disease_treatment[key] != []:
        cleaned_dict[key] = disease_treatment[key]
cleaned_dict

{'breast cancer': ['hormone replacement therapy',
  'undergone subcutaneous mastectomy'],
 'ovarian cancer': ['hormone replacement therapy',
  'undergone subcutaneous mastectomy'],
 'prostate cancer': ['radical prostatectomy and iodine 125 interstitial radiotherapy'],
 'mutated prostate cancer': ['radical prostatectomy and iodine 125 interstitial radiotherapy'],
 'hereditary retinoblastoma': ['radiotherapy'],
 'unstable angina or non-q-wave myocardial infarction': ['roxithromycin'],
 'coronary-artery disease': ['antichlamydial antibiotics'],
 'cerebral palsy': ['hyperbaric oxygen therapy'],
 'primary pulmonary hypertension ( pph )': ['fenfluramines'],
 'cellulitis': ['g-csf therapy', 'intravenous antibiotic treatment'],
 'foot infection': ['g-csf treatment'],
 "early parkinson 's disease": ['ropinirole monotherapy'],
 'sore throat': ['antibiotics'],
 'female stress urinary incontinence': ['surgical treatment'],
 'stress urinary incontinence': ['therapy'],
 'preeclampsia ( proteinuric h

In [48]:
# Converting dictionary to a dataframe
cleaned_df = pd.DataFrame({"Disease":cleaned_dict.keys(),"Treatments":cleaned_dict.values()})
cleaned_df.head()

,Disease,Treatments
0,breast cancer,"[hormone replacement therapy, undergone subcut..."
1,ovarian cancer,"[hormone replacement therapy, undergone subcut..."
2,prostate cancer,[radical prostatectomy and iodine 125 intersti...
3,mutated prostate cancer,[radical prostatectomy and iodine 125 intersti...
4,hereditary retinoblastoma,[radiotherapy]


### Predict the treatment for the disease name: 'hereditary retinoblastoma'

In [49]:
search_item = 'hereditary retinoblastoma'
treatments = cleaned_dict[search_item]
print("Treatments for '{0}' is/are ".format(search_item), end = "")
for i in range(len(treatments)-1):
    print("'{}'".format(treatments[i]),",", end="")
print("'{}'".format(treatments[-1]))

Treatments for 'hereditary retinoblastoma' is/are 'radiotherapy'
